In [199]:
import random
import heapq

class GeneticOptimizatorJSSP:
    # CONSTRUCTOR
    def __init__(self):
        self.jobs = []
        self.number_of_jobs = 0
        self.number_of_machines = 0
        self.solutions = []
        self.solutions_fitness = []
        self.best_solution = []
        self.best_solution_fitness = []

    # GET/SET JOBS
    def _get_number_of_machines(self, jobs):
        _machine_set = set()
        for _job in jobs:
            for _operation in _job:
                _machine_set.add(_operation[0])
        return len(_machine_set)
    
    def get_jobs(self):
        return self.jobs
    
    def set_jobs_from_string(self, jobs):
        _jobs = []
        for _line in jobs.strip().split('\n'):
            _tokens = _line.split()
            _job = []
            for _idx in range(0, len(_tokens), 2):
                _job.append((int(_tokens[_idx]), int(_tokens[_idx+1])))
            _jobs.append(_job)
        self.jobs = _jobs
        self.number_of_jobs = len(_jobs)
        self.number_of_machines = self._get_number_of_machines(_jobs)

    def set_jobs_from_file(self, jobs):
        _jobs = []
        with open(jobs, 'r')  as _file:
            for _line in _file:
                _tokens = _line.split()
                _job = []
                for _idx in range(0, len(_tokens), 2):
                    _job.append((int(_tokens[_idx]), int(_tokens[_idx+1])))
                _jobs.append(_job)
        self.jobs = _jobs
        self.number_of_jobs = len(_jobs)
        self.number_of_machines = self._get_number_of_machines(_jobs)

    # GENETIC ALGORITHM
    def _initialize_population(self, population_size):
        # Output variable
        _population = []

        # Flatten the jobs to operation identifiers int he format (job_index, operation_index)
        _operation_ids = []
        for _job_idx, _job in enumerate(self.jobs):
            for _operation_idx in range(len(_job)):
                _operation_ids.append((_job_idx, _operation_idx))
        
        # Generate random permutations taking in count restrictions
        for _ in range(population_size):
            _not_processed_operations = _operation_ids[:]
            _current_permutation = []
            while len(_not_processed_operations) > 0:
                _valid_operations = []
                for _operation in _not_processed_operations:
                    _job_idx, _operation_idx = _operation
                    if _operation_idx == 0 or ((_job_idx, _operation_idx - 1) in _current_permutation):
                        _valid_operations.append(_operation)
                _selected_operation = random.choice(_valid_operations)
                _current_permutation.append(_selected_operation)
                _not_processed_operations.remove(_selected_operation)
            _population.append(_current_permutation)
        return _population
    
    def _evaluate_fitness(self, population):
        # Output variable
        _fitness = []

        # Calculate for each different solution the makespan
        for _individual in population:
            # Initialize the times for each sequence
            _machine_end_times = [0] * self.number_of_machines
            _job_end_times = [0] * self.number_of_jobs
            for _operation in _individual:
                _job_idx, _operation_idx = _operation
                _machine, _processing_time = self.jobs[_job_idx][_operation_idx]
                # The machine cannot start a new job if the machine is not free or the previous job is not finished
                # To satisfy both constraints the largest time is selected
                _earliest_start = max(_machine_end_times[_machine], _job_end_times[_job_idx])
                # The operation starts at the _earliest_start
                _start_time = _earliest_start
                # The operation ends after the processing time
                _end_time = _start_time + _processing_time
                # Update machine availability
                _machine_end_times[_machine] = _end_time
                # Update job availability
                _job_end_times[_job_idx] = _end_time
            # The maximum end time across all machines to finish defines the total processing time
            _fitness.append(max(_machine_end_times))
        return _fitness
    
    def _selection(self, population, fitness, method='roulette-wheel'):
        if method == 'tournament':
            _c = []
            for _ in range(2):
                # Select a random k individuals
                _k = random.randrange(len(population)) + 1
                # Create a list with the possible indexes for the selection
                _possible_indexes = list(range(len(fitness)))
                # Select _k indexes
                _selected_index = random.choices(_possible_indexes, k=_k)
                # Extract elements at specific indexes
                _selected_fitness = [fitness[i] for i in _selected_index]
                # Select the fittest one
                _max_fitness_value = max(_selected_fitness)
                _max_fitness_index = _selected_fitness.index(_max_fitness_value)
                # Selected chromosomes
                _c.append(population[_selected_index[_max_fitness_index]])
            return (_c[0], _c[1])  
        else:
            # Roulette wheel selection method
            # Initialize the pair of chromosomes selected indexes
            _index_selected_a = 0
            _index_selected_b = 0
            # Calculate the total fitness to obtain the percentages
            _total_fitness = sum(fitness)
            # Calculate the percentages of each chromosome fitness
            _fitness_percentages = [_value / _total_fitness for _value in fitness]
            # Create a list with the possible indexes for the selection
            _possible_indexes = list(range(len(fitness)))
            # Select the index a using the probabilites
            _index_selected_a = random.choices(_possible_indexes, _fitness_percentages, k=1)[0]
            # Delete the selected choice from the list of possible indexes
            _possible_indexes.pop(_index_selected_a)
            _fitness_percentages.pop(_index_selected_a)
            # Select the index b from the remaining possible indexes using the probabilities
            _index_selected_b = random.choices(_possible_indexes, _fitness_percentages, k=1)[0]
            # Return the two corresponding chromosomes to the selected indexes a and b
            return (population[_index_selected_a], population[_index_selected_b])
                
    def _crossover(self, individual_1, individual_2, method='one-point'):
        if method == 'uniform':
            # Intialize the pair of chromosomes for the crossover
            _c_a = individual_1.copy()
            _c_b = individual_2.copy()
            # Iterate over all the operations in the pair of chromosomes
            for _operation_idx, _ in enumerate(_c_a):
                # Swap the operations with a probability of 50%
                if random.random() < 0.5:
                    _operation_1 = _c_a[_operation_idx]
                    _operation_2 = _c_b[_operation_idx]
                    _c_a[_operation_idx] = _operation_2
                    _c_b[_operation_idx] = _operation_1
            return (_c_a, _c_b)
        else:
            # One-point crossover method
            # Intialize the pair of chromosomes for the crossover
            _c_a = individual_1.copy()
            _c_b = individual_2.copy()
            # Select a random position between 1 and the last postion of the chromosome
            _random_position = random.randint(1, len(_c_a) - 1)
            # Swap the parts between the chromosomes
            _c_a[_random_position:], _c_b[_random_position:] = individual_2[_random_position:], individual_1[_random_position:]
            # Rteurn the corresponding chromosomes
            return (_c_a, _c_b)

    def _mutate(self, individual, method, probability=0.5):
        if method == 'flip-probabilistic':
            # Intialize the chromosome for the mutation
            _c = individual.copy()
            # Iterate over all the operations in the pair of chromosomes
            for _operation_idx, _ in enumerate(_c):
                # Swap the operations with a probability of 50%
                if random.random() < probability:
                    # Flip the bits int he chromosome
                    _negated_tuple = tuple(1 - x for x in _c[_operation_idx])
                    # Insert the tuple again
                    _c[_operation_idx] = _negated_tuple
            # Return the mutated chromosome
            return _c
        else:
            # Flip one mutation method
            # Intialize the chromosome for the mutation
            _c = individual.copy()
            # Select a random position between 0 and the last postion of the chromosome
            _random_position = random.randint(0, len(_c) - 1)
            # Flip the bits int he chromosome
            _negated_tuple = tuple(1 - x for x in _c[_random_position])
            # Insert the tuple again
            _c[_random_position] = _negated_tuple
            # Return the mutated chromosome
            return _c

    def _elitism(self, popuplation_1, population_1_fitness, population_2, number=1):
        _new_P_prime = population_2.copy()
        _n_largest = heapq.nsmallest(number, enumerate(population_1_fitness), key=lambda x: x[1])
        _smallest_indexes = [_index for _index, _ in _n_largest]
        for _index in _smallest_indexes:
                _new_P_prime.append(popuplation_1[_index])
        return _new_P_prime

    def _get_best_solution(self, population, fitness):
        _best_solution = []
        _best_solution_fitness = min(fitness)
        _best_solution_index = fitness.index(_best_solution_fitness)
        _best_solution.append(population[_best_solution_index])
        return(_best_solution, _best_solution_fitness)

    def run_genetic_algorithm(self, num_generations, population_size, selection_method='roulette-wheel', crossover_method='one-point', mutation_method='flip-one', mutation_probability=0.5, elitism_number=1):
        # Initialize population P
        _P = self._initialize_population(population_size)

        # Evaluate fitness of all individuals in P
        _F = self._evaluate_fitness(_P)

        # Iterate over generations
        for _ in range(num_generations):
            _P_prime = []
            # Iterate over pairs of individuals
            for _ in range(population_size//2):
                # Selection
                _c_a, _c_b = self._selection(_P, _F, selection_method)
                # Crossover
                _c_prime_a, _c_prime_b = self._crossover(_c_a, _c_b, crossover_method)
                # Mutation
                _c_prime_mutated_a = self._mutate(_c_prime_a, mutation_method, mutation_probability)
                _c_prime_mutated_b = self._mutate(_c_prime_b, mutation_method, mutation_probability)
                # Add to the new population
                _P_prime.append(_c_prime_mutated_a)
                _P_prime.append(_c_prime_mutated_b)
            # Elitism
            _P_prime = self._elitism(_P, _F, _P_prime, elitism_number)
            # Update population
            _P = _P_prime
            # Evaluate fitness of all individuals in P
            _F = self._evaluate_fitness(_P)
        self.solutions = _P
        self.solutions_fitness = _F
        self.best_solution, self.best_solution_fitness = self._get_best_solution(_P, _F)
        

In [204]:
# Create and instance of the genetic optimizator
optimizator = GeneticOptimizatorJSSP()

# Define the problem from a file
optimizator.set_jobs_from_file('problem.txt')

# Define the problme from a string
# problem = """
# 0 88 1 68
# 1 72 0 50
# """
# optimizator.set_jobs_from_string(problem)

# Run genetic algorithm
optimizator.run_genetic_algorithm(10, 4, 'tournament', 'one-point', 'flip-one', elitism_number=2)

print(optimizator.best_solution)
print(optimizator.best_solution_fitness)
print(optimizator.solutions)
print(optimizator.solutions_fitness)


[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9)]
[[(8, 0), (0, 0), (6, 0), (0, 1), (7, 0), (5, 0), (4, 0), (2, 0), (7, 1), (4, 1), (2, 1), (3, 0), (0, 1), (8, 0), (4, 2), (3, 0), (9, 1), (4, 3), (6, 1), (6, 2), (2, 3), (0, 2), (6, 3), (7, 1), (8, 1